Clasificación con insertados

Ejercicio

Entrena un modelo de regresión logística mediante insertados. Imprime el valor de exactitud para el conjunto de entrenamiento.

Calcular los insertados de BERT lleva mucho tiempo (si no estás usando una GPU), por lo que te sugerimos tomar solo 200 elementos aleatorios del conjunto. Los elementos se guardan en el archivo imdb_reviews_200.tsv. Divídelos en conjuntos de entrenamiento y prueba en una proporción de 50:50 para una prueba correcta.

El objetivo se almacena en la variable pos.

Así es como puedes dividir las reseñas en conjuntos de entrenamiento y prueba:

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.5, random_state=42)

¡Es correcto!

Los insertados son características poderosas que te permiten construir un modelo bastante útil, incluso con pequeños conjuntos de datos, ya que representan un contexto específico en su espacio vectorial. Sin embargo, necesitarás otros modelos que puedan calcular insertados a partir del conjunto de datos original. ¡Afortunadamente, BERT y otros modelos sofisticados pueden resolverlo por ti!

Inicialización

import numpy as np
import pandas as pd

import torch
import transformers

from tqdm.auto import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

Variables globales

In [ ]:
# Variables globales
# Limits process to 200 samples
max_sample_size = 200

# Load data
df_reviews = pd.read_csv('/datasets/imdb_reviews_200.tsv', sep="\t")

Preprocesamiento para BERT

In [ ]:
# Converts text into tokens that BERT can undestand
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

ids_list = []
attention_mask_list = []
max_length = 512

for input_text in df_reviews.iloc[:max_sample_size]['review']:
    ids = tokenizer.encode(input_text.lower(), add_special_tokens=True, truncation=True, max_length=max_length)
    padded = np.array(ids + [0]*(max_length - len(ids))) # Pad the shorter sequences with zeros so that they all have length 512
    attention_mask = np.where(padded != 0, 1, 0) # Create a mask. assing 1 for real tokens and 0 for paddings
    ids_list.append(padded)
    attention_mask_list.append(attention_mask)

# Obtains insterted
config = transformers.BertConfig.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased')

# por lo general, el tamaño del lote es igual a 100, 
# pero lo podemos configurar en valores más bajos para reducir los requisitos de memoria
batch_size = 25    

# Generates embeddings
embeddings = []

# Batch processing: Process data in batches of 25 to save memory
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Uso del dispositivo {device}.')
model.to(device)

# Extract token embedding [CLS]: [:,0,:] takes the first token from each sequence, 
# which in BERT contains aggregated information of the entire sentence
for i in tqdm(range(len(ids_list) // batch_size)):
    
    ids_batch = torch.LongTensor(ids_list[batch_size*i:batch_size*(i+1)]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask_list[batch_size*i:batch_size*(i+1)]).to(device)

    with torch.no_grad():
        model.eval()
        batch_embeddings = model(ids_batch, attention_mask=attention_mask_batch)
    # Move tensors between CPU/GPU: Optimize performance
    embeddings.append(batch_embeddings[0][:,0,:].detach().cpu().numpy())

    # Output
#     Uso del dispositivo cpu.
# 100%
#  8/8 [12:31<00:00, 94.16s/it]

Model

In [ ]:
features = np.concatenate(embeddings) # Embeddings de BERT (768 dimensiones por muestra)
target = df_reviews.iloc[:max_sample_size]['pos'] # Etiquetas (1=positivo, 0=negativo)

print(features.shape)
print(target.shape)

# Output
#(200, 768)
#(200,)

# Divides model 50:50 for train and test 
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.5, random_state=42)
log_reg = LogisticRegression(max_iter=100)
log_reg.fit(X_train, y_train)

# Accuracy fot train and test
train_acc = log_reg.score(X_train, y_train)
test_acc = log_reg.score(X_test, y_test)

print(train_acc)
print(test_acc)

# Output
# 1.0
# 0.85
